# Imports

In [115]:
# Python code
import typing
from abc import ABC, abstractmethod

# Data sets
from keras.datasets import mnist

# Computation
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import random

# Metrics
from sklearn import metrics

# Load data set

In [116]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("Shape before: ", X_train.shape)
X_train = X_train.reshape(int(X_train.shape[0]), int(X_train.shape[1] * X_train.shape[2]))
print("Shape after: ", X_train.shape)

# reshape X_test as well
X_test = X_test.reshape(int(X_test.shape[0]), int(X_test.shape[1] * X_test.shape[2]))

Shape before:  (60000, 28, 28)
Shape after:  (60000, 784)


# Models

In [117]:
class Model(ABC):
    @staticmethod
    def compute_metrics(y_actual: np.array, y_predicted : np.array):
        accuracy_score = metrics.accuracy_score(y_actual, y_predicted)
        precision_score = metrics.precision_score(y_actual, y_predicted, average=None)
        recall_score = metrics.recall_score(y_actual, y_predicted, average=None)
        f1_score = metrics.f1_score(y_actual, y_predicted, average=None)
        confusion_matrix = metrics.confusion_matrix(y_actual, y_predicted)
        
        return {
            "accuracy_score": accuracy_score,
            "precision_score": precision_score,
            "recall_score": recall_score,
            "f1_score": f1_score,
            "confusion_matrix": confusion_matrix
        }

    def __init__(self):
        pass
    
    @abstractmethod
    def load(self, X_train: np.array, y_train: np.array):
        pass
    
    @abstractmethod
    def train(self):
        pass
    
    @abstractmethod
    def evaluate(self, X_test: np.array, y_test: np.array):
        pass
    
    @abstractmethod
    def print_metrics(self):
        pass

In [118]:
class DummyClassifier(Model):
    def __init__(self, kwargs):
        self.kwargs = kwargs
    
    def load(self, X_train: np.array, y_train: np.array):
        self.X_train = X_train
        self.y_train = y_train
        self.number_of_classes = tf.keras.utils.to_categorical(y_train).shape[1]

    def train(self):
        pass
    
    def evaluate(self, X_test: np.array, y_test: np.array):
        y_predicted = [random.choice(range(self.number_of_classes)) for _ in range(y_test.size)] 
        self.metrics = Model.compute_metrics(y_test, y_predicted)

    def print_metrics(self):
        if self.metrics:
            print(self.metrics)

In [119]:
class NotDummyClassifier(Model):
    def __init__(self, kwargs):
        self.kwargs = kwargs
        self.actual_instance = RandomForestClassifier()
    
    def load(self, X_train: np.array, y_train: np.array):
        self.X_train = X_train
        self.y_train = y_train
        self.number_of_classes = tf.keras.utils.to_categorical(y_train).shape[1]

    def train(self):
        self.actual_instance.fit(X_train, y_train)
    
    def evaluate(self, X_test: np.array, y_test: np.array):
        y_predicted = self.actual_instance.predict(X_test) 
        self.metrics = Model.compute_metrics(y_test, y_predicted)

    def print_metrics(self):
        if self.metrics:
            print(self.metrics)

# Test

In [120]:
class Test:
    def __init__(self, model_instance : Model):
        self.model_instance = model_instance
        
    def run(self, X_train: np.array, y_train: np.array, X_test: np.array, y_test: np.array):
        self.model_instance.load(X_train, y_train)
        self.model_instance.train()
        self.model_instance.evaluate(X_test, y_test)
        self.model_instance.print_metrics()


In [121]:
Test(\
    DummyClassifier(kwargs=dict())).\
        run(X_train, y_train, X_test, y_test)

{'accuracy_score': 0.0962, 'precision_score': array([0.08587786, 0.08195122, 0.09465021, 0.09414025, 0.10301769,
       0.09567901, 0.1003009 , 0.10275424, 0.0979021 , 0.10683349]), 'recall_score': array([0.09183673, 0.07400881, 0.08914729, 0.0970297 , 0.10081466,
       0.10426009, 0.10438413, 0.09435798, 0.10061602, 0.11000991]), 'f1_score': array([0.0887574 , 0.07777778, 0.09181637, 0.09556314, 0.10190427,
       0.09978541, 0.10230179, 0.09837728, 0.09924051, 0.10839844]), 'confusion_matrix': array([[ 90, 122, 111,  96,  99,  89,  95,  83,  89, 106],
       [122,  84, 110, 113, 101, 125, 117, 116, 124, 123],
       [ 98, 124,  92, 104, 103,  90,  95, 106, 121,  99],
       [118,  98, 102,  98, 104,  88,  96,  97, 105, 104],
       [109, 104,  88, 110,  99,  88, 111,  86,  92,  95],
       [103,  93,  84, 103,  79,  93,  89,  77,  79,  92],
       [ 92,  93,  87, 106,  79, 103, 100,  97, 100, 101],
       [108, 101, 101, 119,  97, 107,  87,  97,  98, 113],
       [104,  97, 100, 101

In [122]:
Test(\
    NotDummyClassifier(kwargs=dict())).\
        run(X_train, y_train, X_test, y_test)

{'accuracy_score': 0.9698, 'precision_score': array([0.97097097, 0.98943662, 0.96061479, 0.95767717, 0.97341513,
       0.96959459, 0.97703549, 0.97154073, 0.96881497, 0.95712861]), 'recall_score': array([0.98979592, 0.99030837, 0.96899225, 0.96336634, 0.9694501 ,
       0.96524664, 0.97703549, 0.96303502, 0.95687885, 0.95143707]), 'f1_score': array([0.98029308, 0.9898723 , 0.96478534, 0.96051333, 0.97142857,
       0.96741573, 0.97703549, 0.96726917, 0.96280992, 0.95427435]), 'confusion_matrix': array([[ 970,    0,    1,    0,    0,    3,    3,    1,    1,    1],
       [   0, 1124,    2,    3,    0,    2,    2,    0,    1,    1],
       [   6,    1, 1000,    5,    2,    1,    2,    9,    6,    0],
       [   0,    0,   11,  973,    0,    6,    0,    9,    6,    5],
       [   2,    0,    1,    0,  952,    0,    7,    1,    2,   17],
       [   3,    0,    0,   15,    2,  861,    5,    2,    3,    1],
       [   7,    3,    0,    1,    3,    6,  936,    0,    2,    0],
       [   1,  